In [1]:
%pip install gensim
#%pip install torch
%pip install Pillow
%pip install requests
%pip install open_clip_torch
#%pip install torchvision
%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu126

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Looking in indexes: https://download.pytorch.org/whl/cu126
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import open_clip
import os
from txt2png import txt_to_png

# Check available devices
if torch.backends.mps.is_available():
    device = torch.device("mps")
    print("Using device: mps")
elif torch.cuda.is_available():
    device = torch.device("cuda")
    print("Using device: cuda")
else:
    device = torch.device("cpu")
    print("Using device: cpu")

d:\Users\PCL_user\Documents\Rob\ASCII_animation-main\ASCII_animation-main\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using device: cuda


In [24]:
# Load CLIP model and tokenizer
model_name = 'hf-hub:laion/CLIP-ViT-g-14-laion2B-s12B-b42K'
clip_model, preprocess = open_clip.create_model_from_pretrained(model_name)
tokenizer = open_clip.get_tokenizer(model_name)
clip_model = clip_model.to(device)
clip_model.eval()  # Set to evaluation mode

# Define ASCII art dimensions and character set
x_dim = 25
y_dim = 8
output_size = y_dim * x_dim
ascii_chars = " .,:;+*#@$%&0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ-_=|/\\()[]{}"
num_chars = len(ascii_chars)
char_to_index = {char: i for i, char in enumerate(ascii_chars)}
index_to_char = {i: char for i, char in enumerate(ascii_chars)}

# Function to get text embeddings from CLIP
def get_clip_text_embedding(text, model, tokenizer, device):
    """
    Get text embeddings from CLIP model
    """
    with torch.no_grad():
        # Tokenize and encode text
        text_tokens = tokenizer([text])
        text_tokens = text_tokens.to(device)
        text_features = model.encode_text(text_tokens)
        text_features = text_features / text_features.norm(dim=-1, keepdim=True)
    
    return text_features.squeeze(0).cpu()

# Get embedding dimension from a sample text
sample_embedding = get_clip_text_embedding("test", clip_model, tokenizer, device)
embedding_dim = sample_embedding.shape[0]
print(f"CLIP embedding dimension: {embedding_dim}")

class ASCIIArtGenerator(nn.Module):
    def __init__(self, input_size, output_size, num_chars):
        super(ASCIIArtGenerator, self).__init__()
        self.fc1 = nn.Linear(input_size, 1024)
        self.bn1 = nn.BatchNorm1d(1024)
        self.fc2 = nn.Linear(1024, 512)
        self.bn2 = nn.BatchNorm1d(512)
        self.fc3 = nn.Linear(512, 512)
        self.bn3 = nn.BatchNorm1d(512)
        self.fc4 = nn.Linear(512, 256)
        self.bn4 = nn.BatchNorm1d(256)
        self.fc5 = nn.Linear(256, 256)
        self.bn5 = nn.BatchNorm1d(256)
        self.fc6 = nn.Linear(256, 128)
        self.bn6 = nn.BatchNorm1d(128)
        self.fc7 = nn.Linear(128, 128)
        self.bn7 = nn.BatchNorm1d(128)
        self.fc8 = nn.Linear(128, 64)
        self.bn8 = nn.BatchNorm1d(64)
        self.fc9 = nn.Linear(64, 64)
        self.bn9 = nn.BatchNorm1d(64)
        self.fc10 = nn.Linear(64, output_size * num_chars)
        #self.bn7 = nn.BatchNorm1d(32)
        #self.fc8 = nn.Linear(32, 16)
        #self.bn8 = nn.BatchNorm1d(16)
        #self.fc9 = nn.Linear(16, output_size * num_chars)
        self.lrelu = nn.LeakyReLU()
        self.dropout = nn.Dropout(0.2)
        self.num_chars = num_chars
        self.output_size = output_size
        
    def forward(self, x):
        x = self.fc1(x)
        x = self.bn1(x)
        x = self.lrelu(x)
        
        x = self.fc2(x)
        x = self.bn2(x)
        x = self.lrelu(x)
        x = self.dropout(x)
        
        x = self.fc3(x)
        x = self.bn3(x)
        x = self.lrelu(x)
        
        x = self.fc4(x)
        x = self.bn4(x)
        x = self.lrelu(x)
        x = self.dropout(x)
        
        x = self.fc5(x)
        x = self.bn5(x)
        x = self.lrelu(x)
        
        x = self.fc6(x)
        x = self.bn6(x)
        x = self.lrelu(x)
        
        x = self.fc7(x)
        x = self.bn7(x)
        x = self.lrelu(x)
        
        x = self.fc8(x)
        x = self.bn8(x)
        x = self.lrelu(x)
        
        
        x = self.fc9(x)
        x = self.bn9(x)
        x = self.lrelu(x)

        x = self.fc10(x)
        return x.view(-1, self.output_size, self.num_chars)

# Function to generate ASCII art
def generate_ascii_art(model, embedding, temperature=1.4):
    model.eval()
    with torch.no_grad():
        # Make sure embedding is a tensor on the correct device
        if not isinstance(embedding, torch.Tensor):
            raise TypeError("Embedding must be a torch.Tensor")
        
        # Move embedding to the model's device
        device = next(model.parameters()).device
        embedding = embedding.to(device)
        
        # Forward pass
        output = model(embedding.unsqueeze(0))
        
        # Apply temperature scaling
        scaled_logits = output / temperature
        
        # Convert to probabilities
        probs = F.softmax(scaled_logits, dim=2)
        
        # Sample from the distribution
        indices = torch.multinomial(probs.view(-1, num_chars), 1).squeeze(-1)
        
        # Convert indices to ASCII characters
        ascii_grid = ""
        for i in range(y_dim):
            for j in range(x_dim):
                ascii_grid += index_to_char[indices[i * x_dim + j].item()]
            ascii_grid += "\n"
        return ascii_grid

# Function to generate ASCII art from a text prompt
def generate_ascii_art_from_text(model, text, clip_model, tokenizer, temperature=1.0):
    """
    Generate ASCII art from a text prompt using CLIP embeddings
    """
    # Get CLIP embedding for the text
    embedding = get_clip_text_embedding(text, clip_model, tokenizer, device)
    
    # Generate ASCII art using the embedding
    return generate_ascii_art(model, embedding, temperature)

# CLIP loss function
def clip_loss(image_tensor, candidate_labels, true_label_index):
    """
    Calculate CLIP loss between image and text labels
    """
    with torch.no_grad():
        # Encode candidate labels
        text_tokens = tokenizer(candidate_labels)
        text_tokens = text_tokens.to(device)
        text_features = clip_model.encode_text(text_tokens)
        text_features = text_features / text_features.norm(dim=-1, keepdim=True)
        
        # Encode image
        image_features = clip_model.encode_image(image_tensor)
        image_features = image_features / image_features.norm(dim=-1, keepdim=True)
        
        # Calculate similarity
        logits = 100 * image_features @ text_features.T
        
        # Compute cross-entropy loss
        labels = torch.tensor([true_label_index], device=device)
        loss = F.cross_entropy(logits, labels)
        
    return loss

CLIP embedding dimension: 1024


In [27]:
class ASCIIArtDataset(torch.utils.data.Dataset):
    def __init__(self, data_pairs, clip_model, tokenizer, device):
        """
        data_pairs: List of tuples (label_text, ascii_art)
        """
        self.data_pairs = data_pairs
        self.clip_model = clip_model
        self.tokenizer = tokenizer
        self.device = device
        
        # Pre-compute embeddings for efficiency
        self.embeddings = {}
        for label_text, _ in data_pairs:
            if label_text not in self.embeddings:
                self.embeddings[label_text] = get_clip_text_embedding(
                    label_text, clip_model, tokenizer, device
                )
    
    def __len__(self):
        return len(self.data_pairs)
    
    def __getitem__(self, idx):
        label_text, ascii_art = self.data_pairs[idx]
        
        # Get cached CLIP embedding
        word_embedding = self.embeddings[label_text]
        
        # Convert ASCII art to target indices
        target_indices = []
        for char in ascii_art:
            if char == '\n':
                continue
            target_indices.append(char_to_index.get(char, 0))
            
        # Pad or truncate to output_size
        if len(target_indices) < output_size:
            target_indices += [0] * (output_size - len(target_indices))
        else:
            target_indices = target_indices[:output_size]
            
        target_tensor = torch.tensor(target_indices, dtype=torch.long)
        
        return word_embedding, target_tensor, label_text

# Function to train the model
def train_model(model, dataset, candidate_labels, num_epochs, batch_size=1, learning_rate=0.001):
    # Load best so far weights
    try:
        model.load_state_dict(torch.load("bsf_weights.pth"))
    except FileNotFoundError:
        print("No weights to load")
    model.to(device)
    
    dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)
    
    criterion = nn.CrossEntropyLoss()
    
    clip_model, preprocess = open_clip.create_model_from_pretrained('hf-hub:laion/CLIP-ViT-g-14-laion2B-s12B-b42K')
    tokenizer = open_clip.get_tokenizer('hf-hub:laion/CLIP-ViT-g-14-laion2B-s12B-b42K')
    clip_model.eval()
    
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=10, factor=0.5, verbose=True)
    
    # Define initial and final weights for the losses
    initial_char_weight = 0.9
    final_char_weight = 0.3
    initial_clip_weight = 0.1
    final_clip_weight = 0.7
    
    best_loss = float("inf")
    for epoch in range(num_epochs):
        total_loss = 0
        
        # Calculate current weights based on progress through training
        # Linear interpolation between initial and final weights
        progress = min(1.0, epoch / (0.7 * num_epochs))  # Reach final weights at 70% of training
        char_weight = initial_char_weight - progress * (initial_char_weight - final_char_weight)
        clip_weight = initial_clip_weight + progress * (final_clip_weight - initial_clip_weight)
        
        print(f"Epoch {epoch+1}/{num_epochs}, Char Weight: {char_weight:.3f}, CLIP Weight: {clip_weight:.3f}")
        
        for word_emb, target_indices, label_text in dataloader:
            word_emb = word_emb.to(device)
            target_indices = target_indices.to(device)
            
            outputs = model(word_emb)
            
            outputs_flat = outputs.view(-1, num_chars)
            targets_flat = target_indices.view(-1)
            
            char_loss = criterion(outputs_flat, targets_flat)
            
            ascii_art = generate_ascii_art(model, word_emb[0].cpu())
            ascii_png = txt_to_png(ascii_art)
            
            image_tensor = preprocess(ascii_png).unsqueeze(0).to(device)
            
            true_label_index = candidate_labels.index(label_text[0])
            
            clip_loss_val = clip_loss(image_tensor, candidate_labels, true_label_index)
            
            # Apply dynamic weights
            loss = char_loss * char_weight + clip_loss_val * clip_weight
            
            optimizer.zero_grad()
            loss.backward()
            # Optional: add gradient clipping
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
            
            total_loss += loss.item()
        
        avg_loss = total_loss / len(dataloader)
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {avg_loss:.4f}")
        
        # Update learning rate scheduler
        scheduler.step(avg_loss)
        
        if avg_loss < best_loss:
            best_loss = avg_loss
            if epoch >= 10:
                torch.save(model.state_dict(), "bsf_weights.pth")
                print("saved best model")
        
        if (epoch + 1) % 5 == 0 or epoch == 0:
            print(f"Example generation for {candidate_labels[true_label_index]}:")
            print(ascii_art)
    
    return model

# Function to prepare training data
def prepare_training_data():
    def read_file_to_string(file_path):
        with open(file_path, 'r', encoding='utf-8') as file:
            content = file.read()
        return content
    
    data_pairs = []
    label_list = []
    txt_dir = os.path.join("..", "Data", "Text")
    
    for txt_file in os.listdir(txt_dir):
        if txt_file.endswith(".txt"):
            label = txt_file.split('_')[0]
            if label in ["bird", "horse", "frog", "fish", "dolphin", "dog", "cat"]:
                ascii_art_txt = read_file_to_string(os.path.join(txt_dir, txt_file))
                label_list.append(label)
                data_pairs.append((label, ascii_art_txt))
    
    return data_pairs, list(set(label_list))

Here is the training loop

In [28]:
# Initialize model
model = ASCIIArtGenerator(embedding_dim, output_size, num_chars)
print("Model initialized with CLIP embedding dimensions")

# Prepare dataset
data_pairs, label_list = prepare_training_data()
dataset = ASCIIArtDataset(data_pairs, clip_model, tokenizer, device)
print("Dataset prepared with CLIP embeddings")

Model initialized with CLIP embedding dimensions
Dataset prepared with CLIP embeddings


In [30]:

# Train model
trained_model = train_model(model, dataset, label_list, batch_size=256, num_epochs=5000, learning_rate=0.001)

d:\Users\PCL_user\Documents\Rob\ASCII_animation-main\ASCII_animation-main\.venv\lib\site-packages\torch\optim\lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Epoch 1/5000, Char Weight: 0.900, CLIP Weight: 0.100
Epoch 1/5000, Loss: 4.4414
Example generation for dolphin:
6#}#9*;fZ*Z&c1Z)01JvRgmrD
Fbto}o&)+{$Y@9cpw|TxiN_kK
q3{H&5c+oXnPOFo;=c)D:MiLj
g7#FIA0\pFGj)X-3f[ai KK-D
x*A6EE+fVDt\7goqTDUh$z+si
4#E|\aq9-+DN%N*FYZbOUC1L@
yp]w9e\V.}&ttd]FlCmWmG8C.
3luYy2V3l(a;OxOn}$O  lJ[o

Epoch 2/5000, Char Weight: 0.900, CLIP Weight: 0.100
Epoch 2/5000, Loss: 4.6649
Epoch 3/5000, Char Weight: 0.900, CLIP Weight: 0.100
Epoch 3/5000, Loss: 4.0012
Epoch 4/5000, Char Weight: 0.899, CLIP Weight: 0.101
Epoch 4/5000, Loss: 4.0840
Epoch 5/5000, Char Weight: 0.899, CLIP Weight: 0.101
Epoch 5/5000, Loss: 3.9489
Example generation for dog:
(nV/Aq1Xx=&(Y(f\sv=._K0N{
x=\t{f:+Yz.+zF9L{ \ZJBR|*
I./.8MRu%1k_:btH6hZo-zW T
wrdCos||1L+Hm= )AMoS2]vvf
TQdpAzDaBSod Vmx42qsqOEDa
MZxI4aNs(Th8v;R9[%]}{{dQq
vVJhaDla0&A:fR*}FOm@{a0y\
q:T[hGzTKCsp=b}*Nbw&L-dZ-

Epoch 6/5000, Char Weight: 0.899, CLIP Weight: 0.101
Epoch 6/5000, Loss: 4.5914
Epoch 7/5000, Char Weight: 0.899, CLIP Wei

In [21]:
# Test with some words
def load_model(weights_path: str):
    model = ASCIIArtGenerator(embedding_dim, output_size, num_chars)
    try:
        model.load_state_dict(torch.load(weights_path))
        print("Loaded weights from previous training")
    except FileNotFoundError:
        print("No weights to load, starting from scratch")
    return model

def test_model(model):
    test_words = ["cat", "dog", "wolf", "eagle"]
    for word in test_words:
        ascii_art = generate_ascii_art_from_text(model, word, clip_model, tokenizer, temperature=0.8)
        print(f"ASCII art for '{word}':")
        print(ascii_art)
        print()

test_model(trained_model)
test_model(load_model("bsf_weights.pth"))

ASCII art for 'cat':
 /. /,    _     - // \/  
\    ( =      / )       (
        (               .
        _  _             
 __                      
                         
                         
                         


ASCII art for 'dog':
          /   -  _ _   _ 
   (        V/  _ _      
                    _   \
            \        \   
                  _      
                         
                         
                         


ASCII art for 'wolf':
             _ _  ._   __
;       _      \     /   
     --                  
                         
                         
                         
                         
                         


ASCII art for 'eagle':
   ,      /  __     __   
_           .       //   
                         
                         
                         
                         
                         
                         


Loaded weights from previous training
ASCII art for 'cat':
 \  \  \    ) 